In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Obtención de los datos

In [4]:
# Leer los datos
data_2023 = pd.read_csv('../datos_2023.csv', delimiter=';', encoding='latin-1', dtype={'superficie_km2': float, 'Codigo_municipio': str})
data_2020 = pd.read_csv('../datos_2020.csv', delimiter=';', encoding='latin-1', dtype={'superficie_km2': float})

# Selecciona las columnas deseadas
selected_columns = data_2020.columns

# Selecciona las columnas deseadas del DataFrame de municipios
train_data = data_2020[data_2020['Serie'] == 'Municipios'][data_2020['Nombre'] != 'Madrid'].iloc[:, 6:]
test_data = data_2023[data_2023['Serie'] == 'Municipios'][data_2023['Nombre'] != 'Madrid'].iloc[:, 6:]
test_data = test_data[selected_columns.intersection(test_data.columns)]
join_data = data_2023[data_2023['Serie'] == 'Municipios'][data_2023['Nombre'] != 'Madrid'].iloc[:, :6]

C:\Users\ana\AppData\Local\Temp\ipykernel_21196\1068069183.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_data = data_2020[data_2020['Serie'] == 'Municipios'][data_2020['Nombre'] != 'Madrid'].iloc[:, 6:]
C:\Users\ana\AppData\Local\Temp\ipykernel_21196\1068069183.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_data = data_2023[data_2023['Serie'] == 'Municipios'][data_2023['Nombre'] != 'Madrid'].iloc[:, 6:]
C:\Users\ana\AppData\Local\Temp\ipykernel_21196\1068069183.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  join_data = data_2023[data_2023['Serie'] == 'Municipios'][data_2023['Nombre'] != 'Madrid'].iloc[:, :6]


In [5]:
X_train = train_data.drop(columns=['sector'])
y_train = train_data['sector']

In [6]:
# import numpy as np
# from sklearn.linear_model import LogisticRegression

# # 1. Agrupar los datos por sector
# grouped_data = train_data.groupby('sector')

# # 2. Calcular estadísticas descriptivas para cada atributo y sector
# stats_by_sector = grouped_data.agg({
#     'sup_explotaciones_agricolas_total': [np.mean, np.std, np.max, np.min],
#     'sup_total_indust': [np.mean, np.std, np.max, np.min],
#     'paro_primario': [np.mean, np.std, np.max, np.min],
#     'paro_secundario': [np.mean, np.std, np.max, np.min],
#     'paro_terciario': [np.mean, np.std, np.max, np.min],
#     'poblacion_censada': [np.mean, np.std, np.max, np.min]
# })

# # 3. Crear un conjunto de datos para el modelo
# X_train = stats_by_sector.values
# y_train = stats_by_sector.index

# # Entrenar un modelo (usando la regresión logística en este ejemplo)
# model = LogisticRegression()
# model.fit(X_train, y_train)


In [7]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# print(train_data.columns)
# # Calcular la matriz de correlación

# dummy_sector = pd.get_dummies(train_data['sector'])
# train_data_dummy = pd.concat([train_data, dummy_sector], axis=1)
# train_data_dummy = train_data_dummy
# corr_data = train_data_dummy.drop(columns=['sector'], inplace=True)
# correlation_matrix =train_data_dummy.corr()

# # Configurar el tamaño de la figura
# plt.figure(figsize=(12, 10))

# # Crear el mapa de calor de la matriz de correlación
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")

# # Añadir título
# plt.title('Matriz de Correlación de Atributos')

# # Mostrar el plot
# plt.show()


In [8]:
# mask = correlation_matrix.abs() >= 0.8
# reduced_correlation_matrix = correlation_matrix.mask(mask)

# # Configurar el tamaño de la figura
# plt.figure(figsize=(24, 20))

# # Crear el mapa de calor de la matriz de correlación
# sns.heatmap(reduced_correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")

# # Añadir título
# plt.title('Matriz de Correlación de Atributos')

# # Mostrar el plot
# plt.show()


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer


# Codificar la variable categórica 'sector'
label_encoder = LabelEncoder()
train_data['sector_encoded'] = label_encoder.fit_transform(train_data['sector'])

# Dividir datos en variables independientes y dependiente
X = train_data.drop(['sector', 'sector_encoded'], axis=1)
y = train_data['sector_encoded']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un imputador para reemplazar los valores faltantes con la media
imputer = SimpleImputer(strategy='mean')

# Aplicar el imputador a tus datos de entrenamiento
X_train_imputed = imputer.fit_transform(X_train)


# Entrenamiento del modelo (Random Forest Classifier como ejemplo)
rf_model = RandomForestClassifier()
rf_model.fit(X_train_imputed, y_train)


# Aplicar el mismo imputador a tus datos de prueba
X_test_imputed = imputer.transform(X_test)

# Realizar predicciones en tus datos de prueba imputados
# Validación del modelo
y_pred = rf_model.predict(X_test_imputed)
print(classification_report(y_test, y_pred))


# Preprocesamiento similar al de los datos del 2020
data_predict_imputed = imputer.transform(test_data)

nan_indices = np.isnan(data_predict_imputed).any(axis=0)
columnas_con_nan = np.where(nan_indices)[0]


# Predecir el sector para el año 2023
predictions_2023 = rf_model.predict(data_predict_imputed)

# Decodificar las predicciones si es necesario
predictions_decoded = label_encoder.inverse_transform(predictions_2023)

# Agregar las predicciones al dataframe de 2023
test_data['sector'] = predictions_decoded

test_data.to_csv('prueba.csv', sep=';', index=False)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.97      1.00      0.99        35

    accuracy                           0.97        36
   macro avg       0.49      0.50      0.49        36
weighted avg       0.95      0.97      0.96        36



c:\Users\ana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [10]:
test_data['Nombre'] = data_2023['Nombre']

# Combinar los DataFrames usando el nombre como clave
csv = pd.merge(data_2023, test_data, on=['Nombre'], how='inner')
csv = csv.iloc[:, :6]

# Asigna el sector de test_data a los municipios correspondientes en csv
csv['sector'] = csv['Nombre'].map(test_data.set_index('Nombre')['sector'])

csv.to_csv('cluster_sector.csv', index=False, sep=';', encoding='latin-1')